In [ ]:
import requests
import time
import pandas as pd
from google.colab import drive
import os
import datetime

# Montar Google Drive
drive.mount('/content/drive')

def obtener_datos():
    url = "https://api.caesistemas.com.ar/v1/ntr.php"
    params = {"api_key": "MGgUAqeDyzDrmEQawzw7xN68"}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        print("Solicitud exitosa")
        data = response.json()
        estado = data.get("estado", {})
        codigo = estado.get("codigo")

        if codigo == -2:
            print("Esperando...")
            mensaje = estado.get("mensaje")
            tiempo_espera = int(mensaje.split(" ")[1])
            print(f"Esperando {tiempo_espera} segundos antes de intentar nuevamente...")
            time.sleep(tiempo_espera)
            return obtener_datos()  # Intenta obtener los datos nuevamente después de esperar
        elif codigo == 0:
            print("Datos disponibles para procesar")
            return data["datos"]["ntr"]
        else:
            mensaje = estado.get("mensaje")
            raise Exception(f"Error en la API: {mensaje}")
    else:
        raise Exception("Error al obtener los datos de la API:", response.status_code)

def main():
    global horometros_anteriores

    # Ruta del archivo CSV
    archivo_csv = '/content/drive/My Drive/horometros_anteriores.csv'

    ntr_data = obtener_datos()

    df_horometros = pd.read_csv(archivo_csv, index_col=0)
    print("Archivo CSV cargado exitosamente.")

    # Utilizar comprensiones de listas para construir alias_list y horometro_list
    alias_list = [item['alias'] for item in ntr_data]
    horometro_list = [item['totalizadores']['horometro'] for item in ntr_data]

    # Crear DataFrame directamente desde los datos
    df = pd.DataFrame({'Alias': alias_list, 'Horometro': horometro_list})

    # Filtrar las filas con alias que comienzan con ": RS", "RTG", "GR" o "FL"
    df_filtrado = df[df['Alias'].str.startswith(('RS', 'RTG', 'GR', 'FL'))]

     # Ordenar el DataFrame por el alias
    df_filtrado = df_filtrado.sort_values(by='Alias')

    for index, row in df_filtrado.iterrows():
        alias = row['Alias']
        horometro_actual = row['Horometro']
        if alias.startswith('FL') or alias == 'RS 39' or alias == 'RS 40':
          if alias in df_horometros.index:
              horometro_anterior = df_horometros.loc[alias, 'horometro']
              timestamp_anterior = df_horometros.loc[alias, 'timestamp']
              # Calcular la diferencia de tiempo
              tiempo_transcurrido = datetime.datetime.now().timestamp() - timestamp_anterior
              print("Tiempo transcurrido:", tiempo_transcurrido)
              if tiempo_transcurrido >= 604800:  # 7 días en segundos
                  # Calcular la diferencia de horas
                  diferencia = horometro_actual - horometro_anterior
                  print(f"Alias: {alias}, Horometro semana anterior: {horometro_anterior}, Horometro semana actual: {horometro_actual}, Diferencia: {diferencia}")
                  # Actualizar el horómetro y el timestamp para la próxima semana
                  df_horometros.at[alias, 'horometro'] = horometro_actual
                  df_horometros.at[alias, 'timestamp'] = datetime.datetime.now().timestamp()
          else:
               # Si el alias no está en df_horometros, añadirlo con el horómetro actual y el timestamp actual
               df_horometros.at[alias, 'horometro'] = horometro_actual
               df_horometros.at[alias, 'timestamp'] = datetime.datetime.now().timestamp()

    # Guardar el DataFrame df_horometros actualizado en el archivo CSV
    df_horometros.to_csv(archivo_csv)

    print("Proceso completado.")


    # Guardar el DataFrame filtrado como un archivo Excel
    #df_filtrado.to_excel('horometros_Equipos.xlsx', index=False)
    #print("Archivo 'horometros_Equipos.xlsx' guardado correctamente.")
    print(df_filtrado)

if __name__ == "__main__":
    main()

Mounted at /content/drive
Solicitud exitosa
Datos disponibles para procesar
Archivo CSV cargado exitosamente.
Tiempo transcurrido: 606584.406719923
Alias: FL 102, Horometro semana anterior: 2664.2, Horometro semana actual: 2690.1, Diferencia: 25.90000000000009
Tiempo transcurrido: 606584.4053020477
Alias: FL 103, Horometro semana anterior: 794.4, Horometro semana actual: 802.4, Diferencia: 8.0
Tiempo transcurrido: 606584.404458046
Alias: FL 104, Horometro semana anterior: 10516.6, Horometro semana actual: 10518.4, Diferencia: 1.7999999999992724
Tiempo transcurrido: 606584.4035348892
Alias: FL 105, Horometro semana anterior: 6626.7, Horometro semana actual: 6656.4, Diferencia: 29.699999999999818
Tiempo transcurrido: 606584.4074831009
Alias: FL 106, Horometro semana anterior: 562.9, Horometro semana actual: 587.3, Diferencia: 24.399999999999977
Tiempo transcurrido: 606584.4055609703
Alias: FL 107, Horometro semana anterior: 8438.1, Horometro semana actual: 8438.4, Diferencia: 0.299999999

In [1]:
import requests
import time
import pandas as pd
import datetime

# Dentro de la función main():
current_time = datetime.datetime.now()

# Definir la variable global para almacenar los horómetros anteriores
horometros_anteriores = {}

def obtener_datos():
    url = "https://api.caesistemas.com.ar/v1/ntr.php"
    params = {"api_key": "MGgUAqeDyzDrmEQawzw7xN68"}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        print("Solicitud exitosa")
        data = response.json()
        estado = data.get("estado", {})
        codigo = estado.get("codigo")

        if codigo == -2:
            print("Esperando...")
            mensaje = estado.get("mensaje")
            tiempo_espera = int(mensaje.split(" ")[1])
            print(f"Esperando {tiempo_espera} segundos antes de intentar nuevamente...")
            time.sleep(tiempo_espera)
            return obtener_datos()  # Intenta obtener los datos nuevamente después de esperar
        elif codigo == 0:
            print("Datos disponibles para procesar")
            return data["datos"]["ntr"]
        else:
            mensaje = estado.get("mensaje")
            raise Exception(f"Error en la API: {mensaje}")
    else:
        raise Exception("Error al obtener los datos de la API:", response.status_code)

def main():
    global horometros_anteriores
    ntr_data = obtener_datos()

    # Utilizar comprensiones de listas para construir alias_list y horometro_list
    alias_list = [item['alias'] for item in ntr_data]
    horometro_list = [item['totalizadores']['horometro'] for item in ntr_data]

    # Crear DataFrame directamente desde los datos
    df = pd.DataFrame({'Alias': alias_list, 'Horometro': horometro_list})

    # Filtrar las filas con alias que comienzan con ": RS", "RTG", "GR" o "FL"
    df_filtrado = df[df['Alias'].str.startswith(('RS', 'RTG', 'GR', 'FL'))]

     # Ordenar el DataFrame por el alias
    df_filtrado = df_filtrado.sort_values(by='Alias')

    # Almacenar el horómetro cada 7 días y calcular la diferencia
    for index, row in df_filtrado.iterrows():
        alias = row['Alias']
        horometro_actual = row['Horometro']
        if alias.startswith('FL') or alias == 'RS 39' or alias == 'RS 40':
            if alias not in horometros_anteriores:
                horometros_anteriores[alias] = {'horometro': horometro_actual, 'timestamp': current_time}
            else:
                # Calcular la diferencia de tiempo
                tiempo_transcurrido = current_time - horometros_anteriores[alias]['timestamp']
                if tiempo_transcurrido >= 7:  # 7 días en segundos
                    horometro_anterior = horometros_anteriores[alias]['horometro']
                    diferencia = horometro_actual - horometro_anterior
                    print(f"Alias: {alias}, Horometro semana anterior: {horometro_anterior}, Horometro semana actual: {horometro_actual}, Diferencia: {diferencia}")
                    # Actualizar el horómetro y el timestamp para la próxima semana
                    horometros_anteriores[alias] = {'horometro': horometro_actual, 'timestamp': current_time}
    print("Proceso completado.")

    # Guardar el DataFrame filtrado como un archivo Excel
    #df_filtrado.to_excel('horometros_Equipos.xlsx', index=False)
    #print("Archivo 'horometros_Equipos.xlsx' guardado correctamente.")
    print(df_filtrado)

if __name__ == "__main__":
    main()


Solicitud exitosa
Datos disponibles para procesar
Proceso completado.
     Alias  Horometro
12  FL 102     2692.4
7   FL 103      816.3
36  FL 104    10518.4
13  FL 105     6682.2
33  FL 106      596.5
14  FL 107     8438.6
3   FL 108      211.5
35  FL 500     4361.4
11  FL 501     6501.5
28   FL 99       14.7
26   FL160     4927.6
27    GR08    37196.8
0     GR09    38939.8
24    GR10    29052.5
50   RS 39      546.4
10   RS 40    40155.9
23   RTG02    47880.5
5    RTG11    62888.3
6    RTG16    55230.5
17   RTG17    60022.2
22   RTG20     6902.0
37   RTG24       51.2
2    RTG26    29535.9
38   RTG31       69.5
